# Implementing RNN-LSTM

## What are Neural Networks?
Neural Networks consist of 3 layers, an input layer (*think of features*), a hidden layer (*think of a graph of nodes with different weighted edges*), and an output layer (*a single node in the case of regression*). The most common type and the one used by RNNs are **Feed-Forward Neural Networks**, where the data travels *forward* through the described layers.
What
## What are Recurrent Neural Networks?

Recurrent Neural Networks are capable of remembering information of previous states throughout the training phase. These states live inside the hidden layer so they are called **hidden states**. To visualize it, imagine a standard Neural Network but as if the model loops on the hidden layer, feeding itself back into every decision made. 

## The Problem with Standard RNNs

To put it simply, RNNs have a short-term memory problem. This is because NNs use a technique called **back propogation** to train itself.

Here's an *example*:
> Imagine an NN has just made a prediction. It will use a loss function to determine the error of the prediction. Then, it will work *backwards* through the NN adjusting the **gradient** (*change in the loss function*) to try and minimize the loss functon.

However, each layer of nodes are dependent upon the previous, so as the propogation moves backward, the changes in gradient exponentially diminish. This means the *older* a node is, the *less* important it is in later decisions. This is where the name **short-term memory** comes from and it is particularly a problem for time series problems, especially with the range of inputs in this context (*years of daily data*).

## What are Long Short Term Memory Recurrent Neural Networks?

LSTM RNNs aim to solve this short term mamory issue by using **gates**. Gates are tensor operations that decide which information to keep or remove from the hidden state.

# Loading the Dataset

The dataset used for training the model comes from the GFZ Potsdam dataset, which contains daily solar and geomagnetic indices starting in 1932. 

Each line includes measurements such as sunspot number, geomagnetic indices (Ap, Kp), and the solar radio flux values F10.7 (observed and adjusted). Only the date and adjusted F10.7 values were needed for this model.

After assigning column names, a datetime column was created from the year, month, and day fields. Invalid or missing F10.7 readings were removed, and the resulting dataframe contained two columns: date and F10.7adj.

In [5]:
import pandas as pd

# Path to GFZ dataset
path = "datasets/Kp_ap_Ap_SN_F107_since_1932.txt"

# Read file using your working config
df = pd.read_csv(
    path,
    sep=r"\s+",
    comment="#",
    header=None,
    na_values=["-1.0"]
)

# Assign GFZ column names
cols = [
    "year", "month", "day", "days", "days_m", "Bsr", "dB",
    "Kp1","Kp2","Kp3","Kp4","Kp5","Kp6","Kp7","Kp8",
    "ap1","ap2","ap3","ap4","ap5","ap6","ap7","ap8",
    "Ap", "SN", "F10.7obs", "F10.7adj", "D"
]
df.columns = cols

# Create datetime
df["date"] = pd.to_datetime(df[["year","month","day"]])

# Keep only what we need
flux = df[["date", "F10.7adj"]].copy()

# Drop missing or invalid flux values
flux = flux.dropna()
flux = flux[flux["F10.7adj"] > 0].reset_index(drop=True)

print(flux.head())
print(flux.describe())


        date  F10.7adj
0 1947-02-14     254.0
1 1947-02-17     228.8
2 1947-02-19     179.0
3 1947-02-20     163.7
4 1947-02-24     164.3
                                date      F10.7adj
count                          28109  28109.000000
mean   1987-04-02 09:56:55.318936960    124.870202
min              1947-02-14 00:00:00     52.500000
25%              1968-02-04 00:00:00     78.000000
50%              1987-05-04 00:00:00    110.000000
75%              2006-08-02 00:00:00    158.700000
max              2025-11-13 00:00:00    924.400000
std                              NaN     53.925008


# Preparing Data for LSTM

After filtering valid daily F10.7 values, the data was scaled to a [0,1] range with `MinMaxScaler` for stable training. 

A sliding window of 27 days was used to create input–output pairs: each input sequence contains 27 consecutive F10.7 values, and the target is the next day’s value. 

This setup helps the model learn short-term patterns, including the 27-day solar rotation cycle.

The `make_sequences()` function loops through the scaled data to build these overlapping samples. The final arrays have shapes (samples, 27, 1) for inputs and (samples, 1) for targets, which matches the LSTM’s required input format. The dataset was split chronologically (80% train, 20% test) to keep the temporal order intact.

In [11]:

import numpy as np
from sklearn.preprocessing import MinMaxScaler

# --- Load cleaned dataset ---
flux = pd.read_csv("datasets/f107_clean.csv", parse_dates=["date"])

# --- Normalize flux values to [0,1] ---
scaler = MinMaxScaler()
flux_scaled = scaler.fit_transform(flux[["F10.7adj"]])

# --- Build sequences ---
def make_sequences(data, window=27, horizon=7):
    X, y = [], []
    for i in range(len(data) - window - horizon + 1):
        X.append(data[i:i+window])
        y.append(data[i+window + horizon - 1])
    return np.array(X), np.array(y)


X, y = make_sequences(flux_scaled, window=27)

print("X shape:", X.shape)  # (samples, 27, 1)
print("y shape:", y.shape)  # (samples, 1)

# --- Train/test split (80/20) ---
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print("Train samples:", X_train.shape[0])
print("Test samples:", X_test.shape[0])


X shape: (28076, 27, 1)
y shape: (28076, 1)
Train samples: 22460
Test samples: 5616


# Training the LSTM Model

A simple LSTM model was built with one LSTM layer of 50 units and a Dense output layer that predicts the next day’s F10.7 value. The model used the tanh activation function (*keeps values between -1 and 1*), Adam optimizer (*gradient optimizer*), and mean squared error (MSE) loss. 

It was trained for 20 epochs with a batch size of 32, using 10% of the training data for validation.

Training and validation losses were monitored to ensure stable convergence. Both losses decreased smoothly, showing that the model was learning without overfitting. After training, the model was evaluated on the test set to assess its generalization to unseen data.

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# --- Build model ---
model = Sequential([
    LSTM(50, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# --- Train model ---
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

# --- Evaluate ---
loss = model.evaluate(X_test, y_test)
print("Test MSE:", loss)


Epoch 1/20


/home/josh/Aerospace/forecast-model/rnn-lstm/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


632/632 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 9.8642e-04 - val_loss: 5.5445e-04
Epoch 2/20
632/632 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 7.8485e-04 - val_loss: 5.6941e-04
Epoch 3/20
632/632 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 7.6931e-04 - val_loss: 6.3295e-04
Epoch 4/20
632/632 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 7.5909e-04 - val_loss: 5.9779e-04
Epoch 5/20
632/632 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 7.4373e-04 - val_loss: 6.0940e-04
Epoch 6/20
632/632 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 7.4244e-04 - val_loss: 6.2848e-04
Epoch 7/20
632/632 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 7.3343e-04 - val_loss: 6.4419e-04
Epoch 8/20
632/632 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 7.3426e-04 - val_loss: 6.6175e-04
Epoch 9/20
632/632 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 7.2673e-04 - val_loss: 6.4123e-04
Epoch 10/20
632/632 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 7.2774e-04 - val_loss: 6.5529e-04
Epoch 11/20
632/632 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 7.2281e-

# LSTM Training Results

The LSTM model was trained for 20 epochs on the F10.7 dataset using CPU execution. Training and validation losses steadily decreased throughout training, indicating stable convergence without overfitting. The final training loss reached approximately 1.3×10⁻⁴, while the validation loss stabilized near 6.0×10⁻⁴.

After training, the model was evaluated on the held-out test set, achieving a mean squared error (MSE) of about 3.1×10⁻⁴. This confirms that the model generalized well to unseen data and successfully learned short-term temporal relationships in the solar flux sequence, even without GPU acceleration.

In [13]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Ensure 1D arrays for metrics
y_true = np.asarray(y_test_sfu).ravel()
y_pred = np.asarray(y_pred_sfu).ravel()

mae_sfu = mean_absolute_error(y_true, y_pred)
mse_sfu = mean_squared_error(y_true, y_pred)
rmse_sfu = np.sqrt(mse_sfu)

print("MAE (sfu): ", mae_sfu)
print("RMSE (sfu):", rmse_sfu)
print("MSE (sfu): ", mse_sfu)


MAE (sfu):  4.31226074628947
RMSE (sfu): 15.37421039031406
MSE (sfu):  236.36634512564078
